In [35]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","RColorBrewer","zoo","lubridate","tidyverse",
              "readxl","gridExtra","MASS","ggpubr", "mixdist", "MASS")
for(x in libraries) {library(x, character.only=TRUE, warn.conflicts=FALSE, quietly=TRUE)}

theme_set(theme_bw())
version$version.string

options(scipen=10000)

[1] "R version 4.2.2 (2022-10-31 ucrt)"

# Settings

In [36]:
readRDS("backproj_cases_non_Omi_MA.rds") %>% 
filter(onset >= as.Date("2020-01-15") & onset <= as.Date("2021-12-31")) -> temp
temp %>% mutate(t=0:(nrow(temp)-1)) -> backproj_Delta

readRDS("backproj_cases_Omi_MA.rds") %>% 
filter(onset >= as.Date("2020-01-15") & onset <= as.Date("2022-06-30")) -> temp
temp %>% mutate(t=0:(nrow(temp)-1)) -> backproj_Omicron

In [37]:
readRDS("RDS/immune_total_Delta_wDate.RDS") -> immune_total_Delta
readRDS("RDS/immune_total_Omi_wDate.RDS") -> immune_total_Omi

In [38]:
## implementation time of major NPIs
NPI <- as.Date("2021-10-15") ## announcing the gradual ease of the social distancing countermeasures
NPI_0 <- as.Date("2021-11-01") ## Start of the gradual ease of the social distancing countermeasures
NPI_1 <- as.Date("2021-12-06") ## state of emergency
NPI_2 <- as.Date("2022-02-19") ## extending the restricted operation hour(21 -> 22)
NPI_3 <- as.Date("2022-03-05") ## extending the restricted operation hour (22 -> 23)
NPI_4 <- as.Date("2022-04-04") ## extending the restricted operation hour (23 -> 24)
NPI_5 <- as.Date("2022-04-18") ## lifting all interventions

## period for the stepwise estimation
period <- 14

## Moving average
MA=7

# Relative changes in Delta period

In [39]:
################################### DELETE
niter=1000
################################### DELETE

In [40]:
colnames(immune_total_Delta) <- c(paste0(c("immune_per"),1:1000), "t", "onset")

merge(backproj_Delta, immune_total_Delta, by=c("onset"), all.x=TRUE) %>% na.omit() -> dt.backproj

dt.backproj %<>% mutate(t=0:(nrow(dt.backproj)-1))

In [41]:
## generation interval (Nishiura, et al, 2020)
gi_fit = list(shape=2.305, scale=5.452)
generation <- function(t){pweibull(t, shape = gi_fit$shape, scale = gi_fit$scale) - 
                          pweibull(t-1, shape = gi_fit$shape, scale = gi_fit$scale)}

In [42]:
library(foreach); library(doParallel);
(ncore <- detectCores(logical = FALSE))

[1] 8

In [43]:
## period setting
NPI_pre = as.numeric(dt.backproj %>% filter(onset >= NPI-period) %>% dplyr::select(t) %>% min())
NPI_start = as.numeric(dt.backproj %>% filter(onset >= NPI-1) %>% dplyr::select(t) %>% min())
NPI_post = as.numeric(dt.backproj %>% filter(onset >= NPI+period-1) %>% dplyr::select(t) %>% min())

NPI_0_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_0-period) %>% dplyr::select(t) %>% min())
NPI_0_start = as.numeric(dt.backproj %>% filter(onset >= NPI_0-1) %>% dplyr::select(t) %>% min())
NPI_0_post = as.numeric(dt.backproj %>% filter(onset >= NPI_0+period-1) %>% dplyr::select(t) %>% min())

NPI_1_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_1-period) %>% dplyr::select(t) %>% min())
NPI_1_start = as.numeric(dt.backproj %>% filter(onset >= NPI_1-1) %>% dplyr::select(t) %>% min())
NPI_1_post = as.numeric(dt.backproj %>% filter(onset >= NPI_1+period-1) %>% dplyr::select(t) %>% min())

In [44]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling <- list() 
final_sampling <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_pre; period_post <- NPI_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_start+1; period_post <- NPI_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling[[g]]
}

stopCluster(myCluster)

In [49]:
do.call(rbind, final_sampling) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample0

saveRDS(est_sample0, "RDS/est_sample0.rds")

In [114]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling1 <- list() 
final_sampling1 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_0_pre; period_post <- NPI_0_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_0_start+1; period_post <- NPI_0_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling1[[g]]
}

stopCluster(myCluster)

In [115]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling2 <- list() 
final_sampling2 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_1_pre; period_post <- NPI_1_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_1_start+1; period_post <- NPI_1_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling2[[g]]
}

stopCluster(myCluster)

In [116]:
do.call(rbind, final_sampling) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample0

In [117]:
do.call(rbind, final_sampling1) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample1

In [118]:
do.call(rbind, final_sampling2) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample2

In [46]:
llk <- function(param){

        period_pre <- NPI_pre; period_post <- NPI_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_start+1; period_post <- NPI_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample0_Re

In [121]:
llk <- function(param){

        period_pre <- NPI_0_pre; period_post <- NPI_0_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_0_start+1; period_post <- NPI_0_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
            Cs[t] = sum(Css)*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample1_Re

In [122]:
llk <- function(param){

    period_pre <- NPI_1_pre; period_post <- NPI_1_start ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)

    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
        Cs[t] = sum(Css)*param[1] 
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total1 <- sum(llk)


    period_pre <- NPI_1_start+1; period_post <- NPI_1_post ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)
    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation(tau)}    
        Cs[t] = sum(Css)*param[1]*param[2]
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total2 <- sum(llk)

    return(llk_total1 + llk_total2)
}

opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample2_Re



 # Estimation effective basic reproduction number during the Omicron wave

In [17]:
colnames(immune_total_Omi) <- c(paste0(c("immune_per"),1:1000), "t", "onset")

merge(backproj_Omicron, immune_total_Omi, by=c("onset"), all.x=TRUE) %>% na.omit() -> dt.backproj

dt.backproj %<>% mutate(t=0:(nrow(dt.backproj)-1))

In [18]:
## generation time distribution (Abbott, 2022)
gi_fit_omi = list(shape=(3.3)^2/(3.5)^2, scale=(3.5)^2/3.3)
generation_omi <- function(t){pgamma(t, shape = gi_fit_omi$shape, scale = gi_fit_omi$scale) - 
                              pgamma(t-1, shape = gi_fit_omi$shape, scale = gi_fit_omi$scale)}

In [19]:
## period setting
NPI_2_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_2-period) %>% dplyr::select(t) %>% min())
NPI_2_start = as.numeric(dt.backproj %>% filter(onset >= NPI_2-1) %>% dplyr::select(t) %>% min())
NPI_2_post = as.numeric(dt.backproj %>% filter(onset >= NPI_2+period-1) %>% dplyr::select(t) %>% min())

NPI_3_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_3-period) %>% dplyr::select(t) %>% min())
NPI_3_start = as.numeric(dt.backproj %>% filter(onset >= NPI_3-1) %>% dplyr::select(t) %>% min())
NPI_3_post = as.numeric(dt.backproj %>% filter(onset >= NPI_3+period-1) %>% dplyr::select(t) %>% min())

NPI_4_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_4-period) %>% dplyr::select(t) %>% min())
NPI_4_start = as.numeric(dt.backproj %>% filter(onset >= NPI_4-1) %>% dplyr::select(t) %>% min())
NPI_4_post = as.numeric(dt.backproj %>% filter(onset >= NPI_4+period-1) %>% dplyr::select(t) %>% min())

NPI_5_pre = as.numeric(dt.backproj %>% filter(onset >= NPI_5-period) %>% dplyr::select(t) %>% min())
NPI_5_start = as.numeric(dt.backproj %>% filter(onset >= NPI_5-1) %>% dplyr::select(t) %>% min())
NPI_5_post = as.numeric(dt.backproj %>% filter(onset >= NPI_5+period-1) %>% dplyr::select(t) %>% min())

In [20]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling3 <- list() 
final_sampling3 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_2_pre; period_post <- NPI_2_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_2_start+1; period_post <- NPI_2_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling3[[g]]
}

stopCluster(myCluster)

In [21]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling4 <- list() 
final_sampling4 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_3_pre; period_post <- NPI_3_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_3_start+1; period_post <- NPI_3_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling4[[g]]
}

stopCluster(myCluster)

In [22]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling5 <- list() 
final_sampling5 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_4_pre; period_post <- NPI_4_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_4_start+1; period_post <- NPI_4_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling5[[g]]
}

stopCluster(myCluster)

In [23]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

final_sampling6 <- list() 
final_sampling6 <- foreach (g = 1:niter, .packages=c("dplyr", "MASS")) %dopar% {
    
    llk <- function(param){

        period_pre <- NPI_5_pre; period_post <- NPI_5_start ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)

        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1] 
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total1 <- sum(llk)

        
        period_pre <- NPI_5_start+1; period_post <- NPI_5_post ####
        
        Cs <- rep(0, period_post); llk <- rep(0, period_post)
        for (t in period_pre:period_post){
            Css <- rep(0, period_post)
            for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
            Cs[t] = sum(Css)*(1-dt.backproj[(t+1),(g+3)])*param[1]*param[2]
            Cs[Cs<=0] <- 1e-5   
            llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
        llk_total2 <- sum(llk)

        return(llk_total1 + llk_total2)
    }
    
    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)
    
    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, 
            tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> final_sampling6[[g]]
}

stopCluster(myCluster)

In [24]:
do.call(rbind, final_sampling3) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample3

In [25]:
do.call(rbind, final_sampling4) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample4

In [26]:
do.call(rbind, final_sampling5) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample5

In [27]:
do.call(rbind, final_sampling6) -> samples_all
est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample6

In [29]:
llk <- function(param){

    period_pre <- NPI_2_pre; period_post <- NPI_2_start ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)

    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1] 
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total1 <- sum(llk)


    period_pre <- NPI_2_start+1; period_post <- NPI_2_post ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)
    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1]*param[2]
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total2 <- sum(llk)

    return(llk_total1 + llk_total2)
}

opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample3_Re

In [30]:
llk <- function(param){

    period_pre <- NPI_3_pre; period_post <- NPI_3_start ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)

    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1] 
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total1 <- sum(llk)


    period_pre <- NPI_3_start+1; period_post <- NPI_3_post ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)
    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1]*param[2]
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total2 <- sum(llk)

    return(llk_total1 + llk_total2)
}

opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample4_Re

In [31]:
llk <- function(param){

    period_pre <- NPI_4_pre; period_post <- NPI_4_start ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)

    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1] 
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total1 <- sum(llk)


    period_pre <- NPI_4_start+1; period_post <- NPI_4_post ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)
    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1]*param[2]
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total2 <- sum(llk)

    return(llk_total1 + llk_total2)
    }

    opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

    fisher_info <- solve(opt_est$hessian)
    mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample5_Re

In [32]:
llk <- function(param){

    period_pre <- NPI_5_pre; period_post <- NPI_5_start ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)

    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1] 
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total1 <- sum(llk)


    period_pre <- NPI_5_start+1; period_post <- NPI_5_post ####

    Cs <- rep(0, period_post); llk <- rep(0, period_post)
    for (t in period_pre:period_post){
        Css <- rep(0, period_post)
        for (tau in 1:(t-1)){Css[tau] = (dt.backproj$total[t-tau+1])*generation_omi(tau)}    
        Cs[t] = sum(Css)*param[1]*param[2]
        Cs[Cs<=0] <- 1e-5   
        llk[t] <- -(-Cs[t]+dt.backproj$total[t+1]*log(Cs[t])-lgamma(dt.backproj$total[t+1]+1))}
    llk_total2 <- sum(llk)

    return(llk_total1 + llk_total2)
}

opt_est <- optim(c(1,1), fn=llk, method=c("L-BFGS-B"), lower=c(0, 0), control=list(maxit=10000), hessian=TRUE)

fisher_info <- solve(opt_est$hessian)
mvrnorm(n=niter, mu=opt_est$par, Sigma=fisher_info, tol=1e-25, empirical=FALSE, EISPACK=FALSE) -> samples_all

est_sample <- matrix(NA, nrow=2, ncol=3)

options(warn=-1)
for(i in 1:2){
    as.numeric(quantile(samples_all[,i], 0.5)) -> est_sample[i,1]
    as.numeric(quantile(samples_all[,i], 0.025)) -> est_sample[i,2]
    as.numeric(quantile(samples_all[,i], 0.975)) -> est_sample[i,3]
}
options(warn=0)
est_sample -> est_sample6_Re